In [17]:
import pandas as pd

In [18]:
dSet = pd.read_csv('emotions.csv', sep = ';')
dSet=dSet.dropna()
#dSet.dtypes
dSet

,text,source,label
0,C'est une société malhonnête.,fnac.com,0
1,Service client lamentable,fnac.com,0
2,Site éclaté,fnac.com,0
3,A banir la fnac,fnac.com,0
4,A la Fnac d Evry,fnac.com,0
...,...,...,...
991,je vous aime bien et trouve ce dont j'ai beso...,cdiscount.com,1
992,"jamais de fausse notes, même en cas de retour...",cdiscount.com,1
993,satisfait pour la commande,cdiscount.com,1
994,j adore je passe beaucoup de temps a chercher...,cdiscount.com,1


In [19]:
X=dSet.drop(dSet.columns[[1, 2]], axis=1)
X

,text
0,C'est une société malhonnête.
1,Service client lamentable
2,Site éclaté
3,A banir la fnac
4,A la Fnac d Evry
...,...
991,je vous aime bien et trouve ce dont j'ai beso...
992,"jamais de fausse notes, même en cas de retour..."
993,satisfait pour la commande
994,j adore je passe beaucoup de temps a chercher...


In [20]:
Y=dSet[dSet.columns[2]]
Y

0      0
1      0
2      0
3      0
4      0
      ..
991    1
992    1
993    1
994    1
995    1
Name: label , Length: 996, dtype: int64

In [21]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2)

In [22]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification,AutoModelForSequenceClassification
from transformers import pipeline



tokenizer = AutoTokenizer.from_pretrained("tblard/tf-allocine")


In [23]:
train_encodings = tokenizer(X_train['text'].tolist(), truncation=True, padding=True)
#val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(X_test['text'].tolist(), truncation=True, padding=True)

train_encodings

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': [[5, 132, 9870, 14, 459, 335, 258, 4847, 83, 6858, 662, 3953, 60, 23578, 449, 546, 9, 21, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [24]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))

In [26]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=10,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

with training_args.strategy.scope():
    model = TFAutoModelForSequenceClassification.from_pretrained("tblard/tf-allocine")

model.config.id2Label={ 
    "0": "Positive",
    "1": "Negative"
}
model.config.label2id={
    "Positive": 0,
    "Negative": 1
}
trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    #eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()

All model checkpoint layers were used when initializing TFCamembertForSequenceClassification.

All the layers of TFCamembertForSequenceClassification were initialized from the model checkpoint at tblard/tf-allocine.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFCamembertForSequenceClassification for predictions without further training.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


In [27]:
trainer.save_model("C:\\Emotions")